In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder()
    .appName("Capítulo 2")
    .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://EM2021002716.bosonit.local:4041
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1617985782383)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@24c13fe9


# Capítulo 2

    a. Descargar el Quijote 
    https://gist.github.com/jsdario/6d6c69398cb0c73111e49f1218960f79
    
    Aplicar no solo count (para obtener el número de líneas) y show sino probar 
    distintas sobrecargas del método show (con/sin truncate, indicando/sin indicar 
    num de filas, etc) así como también los métodos, head, take, first (diferencias 
    entre estos 3?) 

In [43]:
var quijotePath = "/data/el_quijote.txt"
var quijote = spark.read.textFile(quijotePath)

// Count número de líneas
var lineasQuijote = quijote.count()

// Utilización método show
quijote.show()
quijote.show(10)
quijote.show(5, false) // sin truncar el resultado, muestra 5 líneas, por defecto es true

+--------------------+
|               value|
+--------------------+
|DON QUIJOTE DE LA...|
|Miguel de Cervant...|
|                    |
|       PRIMERA PARTE|
|CAPI?TULO 1: Que ...|
|En un lugar de la...|
|Tuvo muchas veces...|
|En resolucio?n, e...|
|historia ma?s cie...|
|Deci?a e?l, que e...|
|En efecto, remata...|
|Imagina?base el p...|
|linaje y patria, ...|
|Limpias, pues, su...|
|Capi?tulo 2: Que ...|
|Hechas, pues, est...|
|Estos pensamiento...|
|Con estos iba ens...|
|Autores hay que d...|
|muertos de hambre...|
+--------------------+
only showing top 20 rows

+--------------------+
|               value|
+--------------------+
|DON QUIJOTE DE LA...|
|Miguel de Cervant...|
|                    |
|       PRIMERA PARTE|
|CAPI?TULO 1: Que ...|
|En un lugar de la...|
|Tuvo muchas veces...|
|En resolucio?n, e...|
|historia ma?s cie...|
|Deci?a e?l, que e...|
+--------------------+
only showing top 10 rows

+-------------------------------------------------------------------------

quijotePath: String = /data/el_quijote.txt
quijote: org.apache.spark.sql.Dataset[String] = [value: string]
lineasQuijote: Long = 2186


In [33]:
// Métodos head, take y first
quijote.head()  // Sin parámetro devuelve la primera fila

res17: String = DON QUIJOTE DE LA MANCHA


In [32]:
quijote.take(1) // Necesita parámetro y devuelve siempre un Array, aunque el valor sea 1

res16: Array[String] = Array(DON QUIJOTE DE LA MANCHA)


In [28]:
quijote.first() // Devuelve siempre el primer valor, no acepta parámetros

res12: String = DON QUIJOTE DE LA MANCHA


# Ejercicio MnM 

In [49]:
// Cargo el csv en un DataFrame
val mnmDF = spark.read.format("csv")
  .option("header","true")
  .option("inferSchema","true")
  .load("/data/mnm_dataset.csv")

mnmDF.printSchema // compruebo que se ha creado correctamente

// Select del DF los campos "State", "Color" and "Count"
// Agrupa el count de Color por state
// orderBy() desc

val count_mnmDF = mnmDF
  .select("State","Color","Count")
  .groupBy("State","Color")
  .sum("Count")
  .orderBy(desc("sum(Count)"))

count_mnmDF.show(5, false)

// Filtrar por state de California

val caCount_MnmDF = mnmDF
  .select("*")
  .where($"State" === "CA")
  .groupBy("State","Color")
  .sum("Count")
  .orderBy(desc("sum(Count)"))

caCount_MnmDF.show()

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)

+-----+------+----------+
|State| Color|sum(Count)|
+-----+------+----------+
|   CA|Yellow|    100956|
|   WA| Green|     96486|
|   CA| Brown|     95762|
|   TX| Green|     95753|
|   TX|   Red|     95404|
|   CO|Yellow|     95038|
|   NM|   Red|     94699|
|   OR|Orange|     94514|
|   WY| Green|     94339|
|   NV|Orange|     93929|
|   TX|Yellow|     93819|
|   CO| Green|     93724|
|   CO| Brown|     93692|
|   CA| Green|     93505|
|   NM| Brown|     93447|
|   CO|  Blue|     93412|
|   WA|   Red|     93332|
|   WA| Brown|     93082|
|   WA|Yellow|     92920|
|   NM|Yellow|     92747|
+-----+------+----------+
only showing top 20 rows

+-----+------+----------+
|State| Color|sum(Count)|
+-----+------+----------+
|   CA|Yellow|    100956|
|   CA| Brown|     95762|
|   CA| Green|     93505|
|   CA|   Red|     91527|
|   CA|Orange|     90311|
|   CA|  Blue|     89123

mnmDF: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]
count_mnmDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, Color: string ... 1 more field]
caCount_MnmDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, Color: string ... 1 more field]


    b. Del ejercicio de M&M aplicar: 
    
    1. Otras operaciones de agregación como el Max con otro tipo de 
    ordenamiento (descendiente). 
    2. hacer un ejercicio como el “where” de CA que aparece en el libro pero 
    indicando más opciones de estados (p.e. NV, TX, CA, CO). 
    3. Hacer un ejercicio donde se calculen en una misma operación el Max, 
    Min, Avg, Count. Revisar el API (documentación) donde encontrarán 
    este ejemplo: 
    ds.agg(max($"age"), avg($"salary")) 
    ds.groupBy().agg(max($"age"), avg($"salary")) 
    NOTA: $ es un alias de col() 
    4. Hacer también ejercicios en SQL creando tmpView 

In [112]:
// 1. Max 
val maxCountDF = count_mnmDF
    .select("State","sum(Count)")
    .groupBy("State")
    .max("sum(Count)")
    .orderBy(desc("max(sum(Count))"))
maxCountDF.show()


//////////// Prueba Joins
val leftDF = count_mnmDF
    .select("State","Color","sum(Count)")
    .distinct
val rightDF = maxCountDF
val joinDF = leftDF.as("t1").join(rightDF.as("t2"), "State")  // Uso de alias para el where, no haría falta pues las columnas
    .where($"t1.sum(Count)" === $"t2.max(sum(Count))")        // se llama diferentes
    .drop("sum(Count)")  // Quito la columna para que no se repitan valores con los máx
joinDF.show()  // Muestra una tabla con el color que tiene de Máximo en cada State

+-----+---------------+
|State|max(sum(Count))|
+-----+---------------+
|   CA|         100956|
|   WA|          96486|
|   TX|          95753|
|   CO|          95038|
|   NM|          94699|
|   OR|          94514|
|   WY|          94339|
|   NV|          93929|
|   AZ|          92287|
|   UT|          91341|
+-----+---------------+

+-----+------+---------------+
|State| Color|max(sum(Count))|
+-----+------+---------------+
|   WY| Green|          94339|
|   NV|Orange|          93929|
|   WA| Green|          96486|
|   CO|Yellow|          95038|
|   CA|Yellow|         100956|
|   OR|Orange|          94514|
|   NM|   Red|          94699|
|   UT|Orange|          91341|
|   TX| Green|          95753|
|   AZ| Brown|          92287|
+-----+------+---------------+



maxCountDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, max(sum(Count)): bigint]
leftDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, Color: string ... 1 more field]
rightDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, max(sum(Count)): bigint]
joinDF: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]


In [90]:

// 2. Add más States a la condición
val statesCount_MnmDF = mnmDF
  .select("*")
  .where($"State" isin ("CA","NM","AZ")) //isin para filtrar de lista, me ahorro código
  .groupBy("State","Color")
  .sum("Count")
  .orderBy(desc("sum(Count)"))

statesCount_MnmDF.show()

+-----+------+----------+
|State| Color|sum(Count)|
+-----+------+----------+
|   CA|Yellow|    100956|
|   CA| Brown|     95762|
|   NM|   Red|     94699|
|   CA| Green|     93505|
|   NM| Brown|     93447|
|   NM|Yellow|     92747|
|   AZ| Brown|     92287|
|   AZ| Green|     91882|
|   AZ|Orange|     91684|
|   CA|   Red|     91527|
|   NM|Orange|     91251|
|   NM| Green|     91160|
|   AZ|Yellow|     90946|
|   CA|Orange|     90311|
|   NM|  Blue|     90150|
|   AZ|   Red|     90042|
|   AZ|  Blue|     89971|
|   CA|  Blue|     89123|
+-----+------+----------+



statesCount_MnmDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, Color: string ... 1 more field]


In [102]:
// 3. Calcular el máximo, mínimo, count, sum, avg, en la misma consulta
val allMnmDF = mnmDF
    .select("*")
    .groupBy("State","Color")
    .agg(avg("Count"),count("Count"),min("Count"),max("Count"),sum("Count"))
allMnmDF.show()

+-----+------+------------------+------------+----------+----------+----------+
|State| Color|        avg(Count)|count(Count)|min(Count)|max(Count)|sum(Count)|
+-----+------+------------------+------------+----------+----------+----------+
|   WY| Green|55.657227138643066|        1695|        10|       100|     94339|
|   NV|   Red|  55.4944099378882|        1610|        10|       100|     89346|
|   UT|  Blue|54.366767371601206|        1655|        10|       100|     89977|
|   WA|Orange|55.199638118214715|        1658|        10|       100|     91521|
|   NM| Green|  54.1973840665874|        1682|        10|       100|     91160|
|   CA|  Blue| 55.59762944479102|        1603|        10|       100|     89123|
|   WA|   Red| 55.85397965290245|        1671|        10|       100|     93332|
|   NV| Brown| 55.81050090525045|        1657|        10|       100|     92478|
|   AZ| Green| 54.82219570405728|        1676|        10|       100|     91882|
|   CA|   Red| 55.26992753623188|       

allMnmDF: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 5 more fields]


In [125]:
// 4. Utilizar tempViews
mnmDF.createOrReplaceTempView("table_states")
var statesDF = spark.sql("""SELECT State, Color, SUM(Count) as Total FROM table_states 
WHERE State in ('CA','AZ','NM') GROUP BY State, Color ORDER BY SUM(Count) DESC""")
statesDF.show()

+-----+------+------+
|State| Color| Total|
+-----+------+------+
|   CA|Yellow|100956|
|   CA| Brown| 95762|
|   NM|   Red| 94699|
|   CA| Green| 93505|
|   NM| Brown| 93447|
|   NM|Yellow| 92747|
|   AZ| Brown| 92287|
|   AZ| Green| 91882|
|   AZ|Orange| 91684|
|   CA|   Red| 91527|
|   NM|Orange| 91251|
|   NM| Green| 91160|
|   AZ|Yellow| 90946|
|   CA|Orange| 90311|
|   NM|  Blue| 90150|
|   AZ|   Red| 90042|
|   AZ|  Blue| 89971|
|   CA|  Blue| 89123|
+-----+------+------+



statesDF: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]
